# Train Emotion Recognition
ResMaskingNet which is available in PyFeat uses 142.9 million parameters with 74% accuracy.
Human emotion detection only reaches 65+-5%, poor golden standard

3-15 million

In [197]:
import wandb
import random
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import math
import itertools

In [180]:
WANDB_API_KEY = "9ff9cdd1ffada991cc4f457217b7dfa6d0d7ce3b"
WANDB_NOTEBOOK_NAME = "Train Emotion Recognition"
wandb.login()

True

In [189]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(70, layers[0]),
            nn.LeakyReLU(),
            nn.Linear(layers[0], layers[1]),
            nn.LeakyReLU(),
            nn.Linear(layers[1], layers[2]),
            nn.LeakyReLU(),
            nn.Linear(layers[2], layers[3]),
            nn.LeakyReLU(),
            nn.Linear(layers[3], 7),
        )
    
    def forward(self, inputs):
        return self.net(inputs)

In [108]:
class MultiEmoVA(Dataset):
    def __init__(self, df):
        super().__init__()
        self.data = df
        self.inputs = torch.tensor(data.drop(["emotion","path", "index"], axis=1).to_numpy(np.float32))
        self.index2label = [label for label in data["emotion"].unique()]
        label2index = {label: i for i, label in enumerate(self.index2label)}
        self.labels = torch.tensor(data["emotion"].apply(lambda x: label2index[x]))

    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]
    
    def map(self):
        return self.index2label


In [109]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [179]:
data = pd.read_csv("data\DiffusionFER\DiffusionEmotion_S\cropped\labels.csv")

learn, test = train_test_split(data, test_size = 0.1, stratify= data["emotion"])
train, val = train_test_split(learn, test_size = 0.2, stratify= learn["emotion"])

c:\Users\Shuai Shao\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Shuai Shao\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Shuai Shao\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Shuai Shao\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and 

In [195]:
wandb.init(
    project = "Furhat Coach"
)

wandb.config = {
        "learning_rate": 0.001,
        "architecture": (512, 256, 128, 64),
        "epochs": 1000,
        "batch": 257,
    }

###########################################################################################

train_loader = DataLoader(MultiEmoVA(train), batch_size= wandb.config["batch"], shuffle = True)
val_loader = DataLoader(MultiEmoVA(val), batch_size= wandb.config["batch"], shuffle = True)
test_loader = DataLoader(MultiEmoVA(test), batch_size= len(test), shuffle = True)

model = MyNeuralNetwork(wandb.config["architecture"]).to(device)

def init_weights(layer):
    if isinstance(layer, nn.Linear):
        torch.nn.init.kaiming_normal_(layer.weight)

model.net.apply(init_weights)

loss_func = nn.CrossEntropyLoss()

optim = torch.optim.Adam(model.parameters(), lr = wandb.config["learning_rate"])


batch_size = 257

##########################################################################################

for epoch in range(wandb.config["epochs"]):
    train_correct = 0
    train_total = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        output = model(inputs)
        train_loss = loss_func(output, labels)
        train_loss.backward()
        optim.step()
        optim.zero_grad()
        train_correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
        train_total += batch_size

    val_correct = 0
    val_total = 0
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        output = model(inputs)
        val_loss = train_loss = loss_func(output, labels)
        val_correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
        val_total += batch_size
        
    print(f"Training accuracy is: {train_correct/train_total}")
    print(f"Validation accuracy is: {val_correct/val_total}")
    wandb.log({"train_acc": train_correct/train_total, "val_acc": val_correct/val_total,"train_loss": train_loss, "val_loss": val_loss})

test_correct = 0
test_total = 0
for inputs, labels, in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    output = model(inputs)
    test_correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
    test_total += batch_size
print(f"Test accuracy is: {test_correct/test_total}")
wandb.log({"test_acc": test_correct/test_total})

model.to_onnx()
wandb.save("model.onnx")
wandb.finish()

Training accuracy is: 0.16653697192668915
Validation accuracy is: 0.3322957158088684
Training accuracy is: 0.2669260799884796
Validation accuracy is: 0.19922178983688354
Training accuracy is: 0.24980545043945312
Validation accuracy is: 0.16264592111110687
Training accuracy is: 0.21322956681251526
Validation accuracy is: 0.2715953290462494
Training accuracy is: 0.21245136857032776
Validation accuracy is: 0.1447470784187317
Training accuracy is: 0.2894941568374634
Validation accuracy is: 0.08093385398387909
Training accuracy is: 0.2077821046113968
Validation accuracy is: 0.37042802572250366
Training accuracy is: 0.26303502917289734
Validation accuracy is: 0.2871595323085785
Training accuracy is: 0.252140074968338
Validation accuracy is: 0.1424124538898468
Training accuracy is: 0.2669260799884796
Validation accuracy is: 0.13696497678756714
Training accuracy is: 0.24669261276721954
Validation accuracy is: 0.1416342407464981
Training accuracy is: 0.3494163453578949
Validation accuracy is: 0

test_acc,▁
train_acc,▁▅▄▆▆▆▅▆▆▇▆▇▇▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▆██▇▇█████
train_loss,█▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▂▅▆▆▅▆▆▇▆▇▇▅▆▇▆▇▇▇▇▇▇▇▇▇▇███▆▇██▇█████
val_loss,█▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,0.87626
train_acc,0.86926
train_loss,0.39897
val_acc,0.87626
val_loss,0.39897


In [ ]:
class ConfusionMatrix:
    def __init__(self, classlist):
        self.classes = {key: 0 for key in classlist}
        self.large_matrix = {key: 0 for key in itertools.product(classlist, classlist)}
        self.tn = 0
        self.fn = 0
        self.tp = 0
        self.fp = 0
    
    def add(label, result):
        if 
        

            

    

In [194]:
torch.save(model, "models/acc_96.8")
#torch.load

In [196]:
model.to_onnx()
wandb.save("model.onnx")

AttributeError: 'MyNeuralNetwork' object has no attribute 'to_onnx'

In [158]:
correct = 0
total = 0
for inputs, labels, in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    output = model(inputs)
    correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
    print(output.softmax(dim=1).argmax(dim=1))
    print(labels)
    total += batch_size
print(f"Test accuracy is: {correct/total}")

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [186]:
wandb.finish()